--- Day 2: Password Philosophy ---
Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan.

The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen.

To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

For example, suppose you have the following list:

1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, cdefg, is not; it contains no instances of b, but needs at least 1. The first and third passwords are valid: they contain one a or nine c, both within the limits of their respective policies.

How many passwords are valid according to their policies?

Your puzzle answer was 418.

The first half of this puzzle is complete! It provides one gold star: *

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('C:/Users\Loren/Desktop/advent_of_code_day2.txt', 
                sep=':', 
                header=None)

In [3]:
df.head()

,0,1
0,1-4 j,jjjqzmgbjwpj
1,2-4 w,sckwwf
2,1-13 b,bcbbbbbbbbbbhbb
3,10-11 x,xjxxxxxxxxzxxx
4,13-14 d,dddddddddddddd


In [4]:
df = df.rename(columns={0: "policy", 1: "password"}).reset_index()
df.head()

,index,policy,password
0,0,1-4 j,jjjqzmgbjwpj
1,1,2-4 w,sckwwf
2,2,1-13 b,bcbbbbbbbbbbhbb
3,3,10-11 x,xjxxxxxxxxzxxx
4,4,13-14 d,dddddddddddddd


In [5]:
#first split
split_data = list(df["policy"].str.split(" "))
col_names = ["pol_range", "req_char"]
split_df = pd.DataFrame(split_data, columns=col_names).reset_index()
split_df.head()

,index,pol_range,req_char
0,0,1-4,j
1,1,2-4,w
2,2,1-13,b
3,3,10-11,x
4,4,13-14,d


In [6]:
#merge with df
df = df.merge(split_df, on='index')
df.head()

,index,policy,password,pol_range,req_char
0,0,1-4 j,jjjqzmgbjwpj,1-4,j
1,1,2-4 w,sckwwf,2-4,w
2,2,1-13 b,bcbbbbbbbbbbhbb,1-13,b
3,3,10-11 x,xjxxxxxxxxzxxx,10-11,x
4,4,13-14 d,dddddddddddddd,13-14,d


In [7]:
#second split 
split_data = list(df["pol_range"].str.split("-"))
col_names = ["pol_min", "pol_max"]
split_df = pd.DataFrame(split_data, columns=col_names).reset_index()
split_df.head()

,index,pol_min,pol_max
0,0,1,4
1,1,2,4
2,2,1,13
3,3,10,11
4,4,13,14


In [8]:
#merge with df
df = df.merge(split_df, on='index')
df.head()

,index,policy,password,pol_range,req_char,pol_min,pol_max
0,0,1-4 j,jjjqzmgbjwpj,1-4,j,1,4
1,1,2-4 w,sckwwf,2-4,w,2,4
2,2,1-13 b,bcbbbbbbbbbbhbb,1-13,b,1,13
3,3,10-11 x,xjxxxxxxxxzxxx,10-11,x,10,11
4,4,13-14 d,dddddddddddddd,13-14,d,13,14


How many passwords are valid?

In [9]:
def check_pw_valid(pw, req_char, pol_min, pol_max):
    req_char_count = pw.count(req_char)
    if (req_char_count <= int(pol_max)) & (req_char_count >= int(pol_min)):
        return 1
    else:
        return 0
    

df['pw_valid'] = df.apply(lambda x: check_pw_valid(x['password'], x['req_char'], 
                                                   x['pol_min'], x['pol_max']), axis=1)

In [10]:
df['pw_valid'].sum()

418

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 8 columns):
index        1000 non-null int64
policy       1000 non-null object
password     1000 non-null object
pol_range    1000 non-null object
req_char     1000 non-null object
pol_min      1000 non-null object
pol_max      1000 non-null object
pw_valid     1000 non-null int64
dtypes: int64(2), object(6)
memory usage: 70.3+ KB


In [12]:
df['pol_min'] = df['pol_min'].astype(str).astype(int)
df['pol_max'] = df['pol_max'].astype(str).astype(int)

Part 2

--- Part Two ---
While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting.

The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

1-3 a: abcde is valid: position 1 contains a and position 3 does not.
1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.
2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.
How many passwords are valid according to the new interpretation of the policies?

Your puzzle answer was 616.

Both parts of this puzzle are complete! They provide two gold stars: **

In [13]:
def check_pw_valid_v2(pw, req_char, pol_min, pol_max):
    if (pw[pol_min] != req_char) & (pw[pol_max] != req_char):
        return 0 #not valid pw
    elif (pw[pol_min] == req_char) & (pw[pol_max] == req_char):
        return 0 #not valid pw
    elif (pw[pol_min] == req_char):
        return 1 #valid pw
    elif (pw[pol_max] == req_char):
        return 1 #valid pw
    
df['pw_valid_v2'] = df.apply(lambda x: check_pw_valid_v2(x['password'], x['req_char'], 
                                                   x['pol_min'], x['pol_max']), axis=1)

In [14]:
df.head(10)

,index,policy,password,pol_range,req_char,pol_min,pol_max,pw_valid,pw_valid_v2
0,0,1-4 j,jjjqzmgbjwpj,1-4,j,1,4,0,1
1,1,2-4 w,sckwwf,2-4,w,2,4,1,1
2,2,1-13 b,bcbbbbbbbbbbhbb,1-13,b,1,13,1,1
3,3,10-11 x,xjxxxxxxxxzxxx,10-11,x,10,11,0,1
4,4,13-14 d,dddddddddddddd,13-14,d,13,14,1,0
5,5,16-18 s,ksttbjsstpnsvtcjnx,16-18,s,16,18,0,0
6,6,3-8 k,gkkqkbpvkrx,3-8,k,3,8,1,1
7,7,3-7 c,mccnjgcxkfkp,3-7,c,3,7,1,0
8,8,3-7 b,hgbqzrjvwqbfc,3-7,b,3,7,0,1
9,9,8-14 r,rrrrrrrfrrrtrrrr,8-14,r,8,14,1,1


In [15]:
df['pw_valid_v2'].sum()

616